<a href="https://colab.research.google.com/github/Yuji-Sakata0110/nurse_shift_optimization/blob/main/nurse_shift_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 遺伝子最適化アルゴリズムでナースのシフト選びを最適化

## 参考

### 1. 遺伝的アルゴリズムでナーススケジューリング問題（シフト最適化）を解く
https://qiita.com/shouta-dev/items/1970c2746c3c30f6b39e

### 2. 最適化アルゴリズムを実装していくぞ（遺伝的アルゴリズム）
https://qiita.com/pocokhc/items/bca2b374b95c606e110f


In [19]:
import random

# 看護師の数
nurses_count = 5

# 1週間の日数
days_in_week = 7

# シフトの種類
shift_types = ['A', "P", 'N', 'R']

# 各看護師の希望シフト
nurse_preferences = {
    0: ['A', 'A', 'N', 'N', 'R', 'R', 'R'],
    1: ['N', 'N', 'A', 'A', 'A', 'N', 'R'],
    2: ['R', 'R', 'A', 'N', 'N', 'A', 'A'],
    3: ['A', 'A', 'A', 'N', 'N', 'N', 'R'],
    4: ['R', 'R', 'R', 'A', 'A', 'A', 'N']
}

# 適応度関数の定義 制約条件をこの関数で評価し、総合点を返却する （制約条件によって、スコアの重みを変更する）
def fitness_function(shift_schedule):
    total_fitness = 0
    for nurse_id, schedule in shift_schedule.items():
        # 連続する勤務日数の計算
        consecutive_days = 0
        for i in range(days_in_week - 1):
            if schedule[i] != 'R' and schedule[i] == schedule[i+1]:
                consecutive_days += 1
            else:
                consecutive_days = 0
            # 連続する勤務日数が多いほど適応度を減らす
            if consecutive_days > 2:
                total_fitness -= consecutive_days

        # 希望シフトとの一致度を計算
        for i in range(days_in_week):
            if schedule[i] == nurse_preferences[nurse_id][i]:
                total_fitness += 1

        # 週休2日であれば、スコアアップ
        if len([s for s in schedule if s == 'R']) == 2:
            total_fitness += 2

    return total_fitness

# 初期集団の生成 ナース5人の初期シフトの組み合わせを５０種類ランダムに準備する
def generate_initial_population():
    population = []
    for _ in range(population_size):
        schedule = {}
        for nurse_id in range(nurses_count):
            # ランダムなシフトの生成
            schedule[nurse_id] = [random.choice(shift_types) for _ in range(days_in_week)]
        population.append(schedule)
    return population

# 選択 ランダムに遺伝子を2つ選択し、スコアが高い方を交叉に回す
def selection(population):
    selected = []
    for _ in range(population_size):
        # ランダムに2つの個体を選択
        a, b = random.choices(population, k=2)
        selected.append(a if fitness_function(a) > fitness_function(b) else b)
    return selected

# 交叉 2つの遺伝子の中でランダムに要素を入れ替える。前半後半で分割。リストに追加して返却する。
def crossover(selected):
    offspring = []
    for i in range(0, len(selected), 2):
        parent1 = selected[i]
        parent2 = selected[i+1]
        crossover_point = random.randint(1, days_in_week - 1)
        child1 = {nurse_id: parent1[nurse_id][:crossover_point] + parent2[nurse_id][crossover_point:]
                  for nurse_id in range(nurses_count)}
        child2 = {nurse_id: parent2[nurse_id][:crossover_point] + parent1[nurse_id][crossover_point:]
                  for nurse_id in range(nurses_count)}

        offspring.extend([child1, child2])
    return offspring

# 突然変異 １０％の確率で突然変異が起き、要素の一部をランダムに変更する。局所的な最適解になることを避ける。
def mutation(offspring):
    mutated_offspring = []
    for schedule in offspring:
        if random.random() < mutation_rate:
            nurse_id = random.randint(0, nurses_count - 1)
            day = random.randint(0, days_in_week - 1)
            schedule[nurse_id][day] = random.choice(shift_types)
        mutated_offspring.append(schedule)
    return mutated_offspring

# アルゴリズムのパラメータ
population_size = 50
mutation_rate = 0.1
generations = 100

# 初期集団の生成
population = generate_initial_population()

# 最適な解の探索
for generation in range(generations):
    print(f"Generation {generation+1}/{generations}")

    # 選択
    selected = selection(population)

    # 交叉
    offspring = crossover(selected)

    # 突然変異
    mutated_offspring = mutation(offspring)

    # 新しい世代の生成
    population = mutated_offspring

    # 最良なスケジュールの表示
    best_schedule = max(population, key=fitness_function)
    print("Best schedule:", best_schedule)
    print("Fitness:", fitness_function(best_schedule))
    print()

# 最良なスケジュールの表示
best_schedule = max(population, key=fitness_function)
print("Final best schedule:", best_schedule)
print("Final fitness:", fitness_function(best_schedule))


Generation 1/100
Best schedule: {0: ['A', 'A', 'N', 'N', 'A', 'R', 'A'], 1: ['N', 'N', 'N', 'A', 'N', 'R', 'R'], 2: ['A', 'R', 'A', 'N', 'R', 'A', 'N'], 3: ['A', 'R', 'A', 'R', 'A', 'P', 'R'], 4: ['R', 'R', 'R', 'R', 'R', 'P', 'A']}
Fitness: 23

Generation 2/100
Best schedule: {0: ['A', 'A', 'N', 'N', 'A', 'R', 'R'], 1: ['N', 'N', 'N', 'A', 'N', 'R', 'R'], 2: ['A', 'R', 'A', 'N', 'R', 'A', 'P'], 3: ['A', 'R', 'A', 'R', 'A', 'P', 'P'], 4: ['R', 'R', 'R', 'R', 'R', 'P', 'P']}
Fitness: 27

Generation 3/100
Best schedule: {0: ['A', 'A', 'N', 'N', 'A', 'R', 'P'], 1: ['N', 'N', 'N', 'A', 'N', 'P', 'A'], 2: ['A', 'R', 'A', 'N', 'R', 'A', 'N'], 3: ['A', 'R', 'A', 'R', 'A', 'P', 'A'], 4: ['R', 'R', 'R', 'R', 'R', 'P', 'A']}
Fitness: 21

Generation 4/100
Best schedule: {0: ['A', 'A', 'N', 'N', 'R', 'A', 'R'], 1: ['N', 'N', 'N', 'A', 'R', 'P', 'P'], 2: ['A', 'R', 'A', 'N', 'R', 'A', 'N'], 3: ['A', 'R', 'A', 'R', 'A', 'P', 'A'], 4: ['R', 'R', 'R', 'R', 'R', 'N', 'A']}
Fitness: 24

Generation 5/100